# RST contrast with continuous Deformable Mirror (DM)
### Goal

After manage our basic contrast tool, we will try to understand how use our DM and measure his influence on contrast.


## Use DM

In [ ]:
#WebbPSF requierement
%pylab inline --no-import-all
from matplotlib.colors import LogNorm
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'

import os
os.environ['WEBBPSF_PATH'] = '/home/marken/GitHub/webbpsf_data'
import webbpsf
from webbpsf import roman
import ipywidgets
from astropy.io import fits
import astropy.units as u

In [ ]:
import logging

#Get WebbPSF logs
webbpsf.setup_logging()

#Remove useless logs warning
log = logging.getLogger()
mplfm_logger = logging.getLogger('matplotlib.font_manager')
mplcb_logger = logging.getLogger('matplotlib.colorbar')
mplt_logger = logging.getLogger('matplotlib.ticker')
mplbe_logger = logging.getLogger('matplotlib.backends')

mplfm_logger.setLevel(logging.WARNING)
mplcb_logger.setLevel(logging.WARNING)
mplt_logger.setLevel(logging.WARNING)
mplbe_logger.setLevel(logging.WARNING)

In [ ]:
#Create a coronagraph instancy
SPC = roman.CGI(mode='CHARSPC_F770')


#Display our DM
SPC.dm1.display()


In [ ]:
#We can inject amplitude of 1 meter in 2 different actuators
SPC.dm1.set_actuator(11, 11, 1*u.m)
SPC.dm1.set_actuator(35, 35, 1*u.m)

SPC.dm1.display()

In [ ]:
#Display his influence on contrast
SPC.raw_contrast(display=True)

In [ ]:
#We can add all actuators we want inside grid (default 48x48) and replace value like this
SPC.dm1.set_actuator(11, 11, 0)
SPC.dm1.set_actuator(2, 24, 1*u.m)
SPC.dm1.set_actuator(36, 13, 1*u.cm)
SPC.dm1.display()

In [ ]:
#Flat DM
SPC.dm1.flatten()
SPC.dm1.display()

In [ ]:
#Change nb is number of actuator in line or/and column

nb = 12 


DM_grid = roman.CGI(mode='CHARSPC_F770', nbactuator=nb) #nbactuator is optionnal but nbactuator=48 by default

for i in range(nb):
    for j in range(nb):
        DM_grid.dm1.set_actuator(i, j, 1*u.nm)

#Get itermediates wawefront propagation 
_psf, inter = DM_grid.calc_psf(nlambda=1, return_intermediates=True)

grid = inter[3].phase #Get phase at the end of pupil plan

plt.imshow(grid**30) #Put a high power number to show grid

#We show how push actuator destroy our mean contrast inside the DH but our values are too huge for example